In [8]:
import tensorflow as tf
import tensorflow_datasets as tfds

from tensorflow.keras.utils import to_categorical
from tensorflow.keras import regularizers

from tensorflow import keras
from keras.layers import SimpleRNN, Dense, Input, Flatten, Conv2D, MaxPooling2D, Dropout, BatchNormalization, GlobalAvgPool2D
from keras.models import Sequential, Model

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings 
from IPython.display import Image

warnings.filterwarnings('ignore')
%matplotlib inline

SEED = 34

In [16]:
(train_images, train_labels), (test_images, test_labels) = keras.datasets.mnist.load_data()

In [18]:
train_images = train_images[..., np.newaxis]
test_images = test_images[..., np.newaxis]

In [19]:
print(f"train_images: {train_images.shape}")
print(f"train_labels: {train_labels.shape}")

print(f"test__images: {test_images.shape}")
print(f"test_labels: {test_labels.shape}")

train_images: (60000, 28, 28, 1)
train_labels: (60000,)
test__images: (10000, 28, 28, 1)
test_labels: (10000,)


In [5]:
def norm(data):
  max_value = data.max()
  min_value = data.min()

  return (data - min_value) / (max_value - min_value)

In [6]:
train_images = norm(train_images)
test_images = norm(test_images)

In [7]:
train_labels.min(), train_labels.max()

(0, 9)

In [20]:
def get_CNN_model(input_shape, rate):

    model = Sequential([
            Conv2D(filters=64, kernel_size=(3, 3), strides=1, padding='same',
                activation='relu', input_shape=(28, 28, 1), name='conv_1'),
            MaxPooling2D(pool_size=(2, 2), name='pool_1'),
            Conv2D(filters=32, kernel_size=(3, 3), strides=1, padding='same',
                activation='relu', input_shape=input_shape, name='conv_2'),
            MaxPooling2D(pool_size=(2, 2), name='pool_2'),
            Conv2D(filters=16, kernel_size=(3, 3), strides=1, padding='same',
                activation='relu', name='conv_3'),
            BatchNormalization(),           
            Flatten(name='flatten'),
            Dense(units=32, activation='relu', name='dense_1'),
            Dropout(rate),
            Dense(units=10, activation='softmax', name='dense_2')
    ])
    
    return model

In [21]:
Sequential_model = get_CNN_model(train_images[0].shape, 0.5)
Sequential_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv_1 (Conv2D)             (None, 28, 28, 64)        640       
                                                                 
 pool_1 (MaxPooling2D)       (None, 14, 14, 64)        0         
                                                                 
 conv_2 (Conv2D)             (None, 14, 14, 32)        18464     
                                                                 
 pool_2 (MaxPooling2D)       (None, 7, 7, 32)          0         
                                                                 
 conv_3 (Conv2D)             (None, 7, 7, 16)          4624      
                                                                 
 batch_normalization_1 (Batc  (None, 7, 7, 16)         64        
 hNormalization)                                                 
                                                      

In [22]:
Sequential_model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics = ["accuracy"])

In [23]:
Sequential_hist = Sequential_model.fit(train_images, train_labels, validation_data=(test_images, test_labels), epochs=5, verbose=2)

Epoch 1/5
1875/1875 - 21s - loss: 0.3353 - accuracy: 0.8915 - val_loss: 0.0550 - val_accuracy: 0.9821 - 21s/epoch - 11ms/step
Epoch 2/5
1875/1875 - 9s - loss: 0.1521 - accuracy: 0.9530 - val_loss: 0.0356 - val_accuracy: 0.9891 - 9s/epoch - 5ms/step
Epoch 3/5
1875/1875 - 9s - loss: 0.1221 - accuracy: 0.9611 - val_loss: 0.0532 - val_accuracy: 0.9843 - 9s/epoch - 5ms/step
Epoch 4/5


In [ ]:
plt.plot(Sequential_hist.history['accuracy'], label='accuracy')
plt.plot(Sequential_hist.history['loss'], label='loss')
plt.plot(Sequential_hist.history['val_accuracy'], label='val_accuracy')
plt.plot(Sequential_hist.history['val_loss'], label='val_loss')
plt.legend(loc='upper left')
plt.show()

In [ ]:
res = Sequential_model.predict( test_images[3:4] ) 
res.shape

In [ ]:
plt.imshow(test_images[3], cmap="gray")
plt.show()

In [ ]:
plt.bar(range(10), res[0], color='red')
plt.bar(np.array(range(10)) + 0.35, to_categorical(test_labels[3], 10))
plt.show()

In [ ]:
Sequential_model.save("models/CNN_based_MNIST_001.h5")